# AI 101 CNN - Lab 1
## Deploy a local Jupyter Notebook cluster on HPE Ezmeral Runtime Enterprise

### **Lab workflow:**

In this lab:

1. As tenant user, you will deploy a local Jupyter Notebook server on HPE Ezmeral Runtime Enterprise (formerly known as HPE Ezmeral Container Platform) and use it as a code sandbox to get started with Deep Learning Neural Network model.

2. You will then access your local Jupyter Notebook web UI and dive into the typical data scientist work to develop a deep learning model.

#### Initialize the environment:

Let's first define the environment variables needed to execute this lab part.

In [ ]:
#
# environment variables to be verified by the student
#
studentId="student{{ STDID }}" # your Jupyter Notebook student Identifier (i.e.: student<xx>)
username="student{{ STDID }}" # your HPE ERE tenant login credentials - username 
password="{{ PASSSTU }}" # your HPE ERE tenant login credentials - password - Please replace with password provided in your registration

tenantname="K8sMLTenant" #case sensitive
#
# fixed environment variables setup by the HPE ECP lab administrator - Please DO NOT MODIFY!!
#
gateway_host="{{ HPEECPGWNAME }}"
controller_endpoint="${gateway_host}:8080"
Internet_access="{{ JPHOSTEXT }}"

clusterName="jupyter-notebook-${studentId}"
JupyterNotebookApp="cr-cluster-jupyter-notebook.yaml" # the Jupyter Notebook KD App manifest you will deploy to build your model
sc_secret="sc-secret-AI101-students.yaml"
kc_secret="kc-secret-AI101-students.yaml" #the kubeconfig secret object for the tenant user.

echo "your operation context is:" $username "on tenant" $tenantname 
#echo "your REST API endpoint is:" $controller_endpoint

#### Authenticate as a tenant user in the specified tenant:

In [ ]:
sessionlocation=$(curl -k -i -s --request POST "https://${controller_endpoint}/api/v2/session" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' \
--data-raw '{
"name": "'"$username"'",
"password": "'"$password"'",
"tenant_name": "'"$tenantname"'"
}' | grep Location | awk '{print $2}' | tr -d '\r') #we remove any cr that might exist
echo "This is your session location: " $sessionlocation
SessionId=$(echo $sessionlocation | cut -d'/' -f 5) # extract sessionId for later, for logout
echo "This is your session_Id:" $SessionId

#### Get the Kubeconfig file for your tenant working context:

In [ ]:
mkdir -p ~/.kube
ls ~/.kube
rm -f ~/.kube/config
curl -k -s --request GET "https://${controller_endpoint}/api/v2/k8skubeconfig" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json' > ~/.kube/config
#
echo "Kubeconfig file fetched. Your current Kubernetes working context is: "
kubectl config current-context

## Deploying your local Jupyter Notebook cluster

In [ ]:
#Create the encrypted form of the user-specific kubeconfig file 
config=$(cat ~/.kube/config | base64)
kcConfig=$(echo $config | tr -d '\r' | tr -d ' ') #we remove any potential spaces
#
cat > $kc_secret << EOF
apiVersion: v1
kind: Secret
metadata:
  name: kc-secret-ai101-$studentId
  labels:
    kubedirector.hpe.com/secretType: kubeconfig
    kubedirector.hpe.com/username: $studentId
data:
   config: $kcConfig
EOF

#cat $kc_secret

In [ ]:
kubectl apply -f $kc_secret
kubectl apply -f $sc_secret
kubectl apply -f $JupyterNotebookApp

After a few seconds, you should get the response message to your K8s API request: *kubedirectorcluster/Your-instance-name created*.  

## Check the deployment status of the local Jupyter Notebook cluster

You can use the command below to observe the deployment status of the local Jupyter Notebook server.

The virtual cluster status indicates its overall "state" (top-level property of the status object). It should have a value of **"configured"**. 

**Repeat the command below until the kdcluster is in state "configured"**

In [ ]:
kubectl describe kdcluster $clusterName

## Get the local Jupyter Notebook service endpoint

In [ ]:
#
# Getting the service endpoint URL:
#
JupyterAppURL=$(kubectl describe service -l  kubedirector.hpe.com/kdcluster=${clusterName} | grep gateway/8000 | awk '{print $2}')
JupyterAppPort=$(echo $JupyterAppURL | cut -d':' -f 2) # extract the gateway re-mapped port value.
myJupyterApp_endpoint="https://$gateway_host:$JupyterAppPort"
#echo "Your application service endpoint re-mapped port is: "$JupyterAppPort
#echo "Your Intranet application service endpoint is: "$myJupyterApp_endpoint
echo "Your Jupyter Notebook service endpoint URL is: https://"$Internet_access:$JupyterAppPort
echo "Your local Jupyter Notebook web UI login credentials are: $username / $password"
#

## Connect to your local Jupyter Notebook web UI
Click the service endpoint URL above to connect to your Jupyter Notebook server.
This opens a Jupyter Notebook login screen in a new browser tab. **Use the login credentials from step above to authenticate.** 

>**Important Note:** Once logged in to your local Jupyter Notebook server, navigate to folder ***/source_control/sc-secret-ai101-\<YourStudentId\>/***, then go to ***AI101-CNN*** directory to access the A1 101 CNN notebook: ***MNIST_CNN_Hack_Shack.ipynb***.

> <font color="red"> **Note:** On Windows PC, ***Firefox*** is the recommended browser to connect to your local Jupyter Notebook UI. When using **Chrome** you may observe a message _"Server Not Running"_, in which case just click Restart button.</font>

> <font color="red"> **Note:** If you are seeing a security warning about the certificate while connecting to the local Jupyter Notebook web UI, please accept the risk and proceed to continue.</font>

Enjoy your learning experience about AI 101 Convolutional neural network (CNN)!

### **Time to go through some cleanup**

Delete your secret and your local Jupyter Notebook cluster

In [ ]:
kubectl delete -f $kc_secret
rm -f $kc_secret
kubectl delete -f $sc_secret
kubectl delete -f $JupyterNotebookApp

In [ ]:
curl -k -i -s --request DELETE "https://${controller_endpoint}/api/v2/session/${SessionId}" \
--header "X-BDS-SESSION: $sessionlocation" \
--header 'Accept: application/json' \
--header 'Content-Type: application/json'

Thank you!!!